# Pricing Automation Detection

In [14]:
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import coint

from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px

import numbers
import statistics as st

df = pd.read_csv('final_csv_task1.csv', usecols=['comp_date',
                                             'product_id',
                                             'seller_id',
                                             'min',
                                             'price',
                                             'max' ,
                                             'difference'])
enc = LabelEncoder()
df['enc_comp_date'] = enc.fit_transform(df['comp_date'])
df["comp_date"] =  pd.to_datetime(df["comp_date"], format = "%Y/%m/%d")
df["quarter"] = df.comp_date.dt.quarter

In [15]:
product_id_input = int(input("What's the product id you want to investigate? (Please provide eg. 110064"))
quarter_input = int(input("Which quarter you want to investigate? (Please provide 1, 2, 3 or 4)"))

In [16]:
def pivot(product_id, quarter):
    df_1 = df[(df.product_id == product_id) & (df.quarter == quarter)]
    table = pd.pivot_table(df_1, values='price', index=['comp_date'], columns=['seller_id'])
    table.fillna(method='ffill', inplace=True)
    return table

dfp = pivot(product_id_input, quarter_input)

In [17]:
dfp.reset_index('comp_date', drop = True, inplace = True)

In [18]:
dfp.fillna("null", inplace = True)

In [19]:
dizionario = dict()

for sell in dfp:
    lista_seller = []
    for i in range(1, len(dfp)):

        if dfp.loc[i, sell] != dfp.loc[i-1, sell] and (dfp.loc[i, sell] and dfp.loc[i-1, sell]) != 'null':
            lista_seller.append(i)
    dizionario[sell] = lista_seller

In [20]:
dict_all = dict()

for i in dizionario:
    if len(dizionario[i]) != 0:
        dict_diff = dict()
        
        for j in dizionario: 
            if j != i:
                if len(dizionario[j]) != 0:
                    lista_diff = []
                    for elem_1 in range(len(dizionario[i])):
                        list_temp = [item for item in dizionario[j] if item <= dizionario[i][elem_1]]
                        if len(list_temp) != 0:
                            diff = dizionario[i][elem_1] - max(list_temp)
                            lista_diff.append(diff)
                        else: lista_diff.append('no recent date')
                        
                        dict_diff[j] = lista_diff
        dict_all[i] = dict_diff

df = pd.DataFrame(0, index=list(dict_all.keys()), columns=list(dict_all.keys()))

for col in df.columns:
    for key1,v1 in dict_all.items():
        if col != key1:
            for key2,v2 in v1.items():
                st_dv = st.stdev([x for x in v2 if isinstance(x, numbers.Number)])
                if st_dv <= 1:
                    st_dv = 'A'
                else:
                    st_dv = 'B'
                df.loc[key1, key2] = st_dv